In [1]:
import pandas as pd 
df = pd.read_csv("C:\\Users\\User\\Downloads\\summary_df.csv")

In [2]:
df

,Unnamed: 0,summary,name,category,交通,才藝族,中式餐廳族,科技嘗鮮族,音樂,文創族,支付,有機族,舞蹈,影音串流
0,0,Netflix 是什麼？ Netflix 是串流服務，透過數千種可連結網路的行動裝置，提供各...,Netflix,"['科技嘗鮮族', '影音串流']",False,False,False,True,False,False,False,False,False,True
1,1,SUBSCRIBE: http://bit.ly/29qBUt7 About Netflix...,Netflix,"['科技嘗鮮族', '影音串流']",False,False,False,True,False,False,False,False,False,True
2,2,"33M Followers, 1146 Following, 7069 Posts - Se...",Netflix,"['科技嘗鮮族', '影音串流']",False,False,False,True,False,False,False,False,False,True
3,3,"Netflix stands w/ innovators, large & small, t...",Netflix,"['科技嘗鮮族', '影音串流']",False,False,False,True,False,False,False,False,False,True
4,4,Netflix 。 87984354 個讚· 9813 人正在談論這個。 Welcome t...,Netflix,"['科技嘗鮮族', '影音串流']",False,False,False,True,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1834,1834,2020年7月31日 — 8月1日是「原住民族日」，為了紀念正名，新北市政府「原住民族行政局...,豪日文創,['文創族'],False,False,False,False,False,True,False,False,False,False
1835,1835,佐賀長崎3日遊 · HOME · 觀光資訊 · 豪斯登堡. Follow us.,豪日文創,['文創族'],False,False,False,False,False,True,False,False,False,False
1836,1836,特典墊板SHINee Winter wonderland 日文專輯日版日專溫流鐘鉉KEY 珉...,豪日文創,['文創族'],False,False,False,False,False,True,False,False,False,False
1837,1837,除於學術上有卓越成就外，於臺大與政大任教其間，亦栽培許多宋史研究人才。 1978年5月15日...,豪日文創,['文創族'],False,False,False,False,False,True,False,False,False,False


In [3]:
len(df['name'].unique())

98

In [4]:
from sklearn.model_selection import train_test_split

# Extracting the relevant columns
text_data = df['summary'].values
label_data = df.iloc[:, 4:].values  # Taking only the one-hot encoded labels

# Splitting the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(text_data, label_data, test_size=0.2, random_state=42)

X_train[:5], y_train[:5]

(array(['統編, 72519637. 公司狀態, 核准設立. 公司名稱, 福來商行. 資本總額(元), 200,000. 負責人, 許斯評. 登記地址, 看地圖 臺中市沙鹿區鹿峰裡中山路576之3號1樓 ...',
        '花蓮縣壽豐鄉農民-鄒仁方. 有機短期葉菜、 有機玉女番茄、 有機玉米、 有機玉米筍、 有機美濃瓜. 5. 苗栗縣-小腳丫有機農場. 小黃瓜.玉米筍.紫地瓜.甜菜根.',
        '最受歡迎的山葉鋼琴教材系列. 財團法人山葉音樂振興基金會. 快樂的鍵盤教本 1. 【內容簡介】. 初學鋼琴鍵盤課常用教本，內有美麗的插圖以及非常大的音符，讓小朋友視力 ...',
        '【LINE熱點】博愛生活文創, 手工藝材料行, 地址: 桃園市桃園區博愛路81號，電話: 03 336 0122。全臺必吃美食、熱門景點推薦、美食外送、優惠折扣、線上預約掛號、叫 ...',
        '2023年2月1日 — 越夜越美麗，一起Happy 兔gether ... 邀請大家一起“夜奔松菸”體驗最靚的臺北風格夜文化。 ... 夏季活動#夜間營運#設計點#松山文創園區#松菸夜生活 ...'],
       dtype=object),
 array([[False, False, False, False, False, False, False,  True, False,
         False],
        [False, False, False, False, False, False, False,  True, False,
         False],
        [False,  True, False, False,  True, False, False, False, False,
         False],
        [False, False, False, False, False,  True, False, False, False,
         False],
        [False, False, False, False, False,  True, False, False, False,
         False]]))

In [5]:
#! pip install accelerate -U

In [6]:
from transformers import BertTokenizer
import torch

# Tokenization
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True, max_length=256)
val_encodings = tokenizer(X_val.tolist(), truncation=True, padding=True, max_length=256)

# Create PyTorch Dataset
class SummaryDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float32)
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SummaryDataset(train_encodings, y_train)
val_dataset = SummaryDataset(val_encodings, y_val)


In [7]:
from transformers import BertPreTrainedModel, BertModel
from torch.nn import BCEWithLogitsLoss

class CustomBertForMultiLabelClassification(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.bert = BertModel(config)
        self.classifier = torch.nn.Linear(config.hidden_size, self.config.num_labels)
        self.loss_fct = BCEWithLogitsLoss()

    def forward(self, input_ids, attention_mask=None, token_type_ids=None, labels=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooled_output = outputs[1]
        logits = self.classifier(pooled_output)
        
        loss = None
        if labels is not None:
            loss = self.loss_fct(logits.view(-1, self.num_labels), labels.view(-1, self.num_labels))

        return loss, logits


In [10]:
from transformers import TrainingArguments, Trainer

model = CustomBertForMultiLabelClassification.from_pretrained('bert-base-chinese', num_labels=y_train.shape[1])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Move the model to the device
model.to(device)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir='./logs'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

trainer.train()


Some weights of CustomBertForMultiLabelClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\User\anaconda3\envs\ml\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/552 [00:00<?, ?it/s]

In [ ]:
results = trainer.evaluate()
